In [196]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [197]:
url = "https://www.imdb.com/search/title/?title_type=feature&primary_language=hi&sort=num_votes,desc"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [198]:
response

<Response [200]>

In [199]:
print(soup)


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film,
Hindi
(Sorted by Number of Votes Descending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?title_type=feature&amp;primary_language=hi" rel

In [200]:
Title = []
# IMDB_ID = []
Date_of_release = []
Genre = []
Cast = []
Director = []
Plot_Summary = []
# Plot_keywords = []
IMDB_rating = []
IMDB_Votes = []
IMDB_Gross = []
# Reviews = []

In [201]:
movie_data = soup.find_all('div', attrs={'class':'lister-item mode-advanced'})

In [202]:
movie_data

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt10811166"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt10811166/"> <img alt="The Kashmir Files" class="loadlate" data-tconst="tt10811166" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzJkZDIwYTAtMGU4Mi00NzU3LWI1MWItODg0M2Q1NmIxYmNlXkEyXkFqcGdeQXVyMTIyNzY0NTMx._V1_UY98_CR5,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt10811166/">The Kashmir Files</a>
 <span class="lister-item-year text-muted unbold">(2022)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">A</span>
 <span class="ghost">|</span>
 <span class="runtime">170 min</span>
 <span class="ghost">|</span>
 <span class="gen

In [203]:
for store in movie_data:
    name = store.h3.a.text # the text is stored
    # storing all movie name in title
    Title.append(name)
    
    release_date = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(', "").replace(')','') # the year is stored here
    Date_of_release.append(release_date)
    
    rate = store.find('div', class_="inline-block ratings-imdb-rating").text.replace('\n', ' ')
    IMDB_rating.append(rate)
    
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    IMDB_Votes.append(vote)
    
    gross = value[1].text if len(value)>1 else '****'
    IMDB_Gross.append(gross)
    
    summary = store.find_all('p', class_="text-muted")
    summary_ = summary[1].text.replace('\n', '') if len(summary) >1 else '*****'
    Plot_Summary.append(summary_)
    
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Director.append(cast[0])
    Cast.append([x.strip() for x in cast[1].split(",")])
    
    genre = store.find('span', class_ = "genre").text.replace('(', "").replace(')','').replace("\n", " ").strip()
    Genre.append(genre)

In [204]:
# creating a dataframe 

movie_DF = pd.DataFrame({'Name of movie':Title, "Year of release": Date_of_release,"Genre":Genre,
                        "Movie Rating":IMDB_rating, "Votes":IMDB_Votes, "Gross":IMDB_Gross,
                        "Summary":Plot_Summary, "Director":Director, "Cast":Cast})

In [205]:
movie_DF.head()

,Name of movie,Year of release,Genre,Movie Rating,Votes,Gross,Summary,Director,Cast
0,The Kashmir Files,2022,Drama,8.3,"548,107",****,"The Kashmir Files' is a story, based on video ...",Vivek Agnihotri,"[Mithun Chakraborty, Anupam Kher, Darshan Kuma..."
1,3 Idiots,2009,"Comedy, Drama",8.4,"387,084",$6.53M,Two friends are searching for their long lost ...,Rajkumar Hirani,"[Aamir Khan, Madhavan, Mona Singh, Sharman Joshi]"
2,Taare Zameen Par,2007,"Drama, Family",8.3,"188,961",$1.22M,An eight-year-old boy is thought to be a lazy ...,"Directors:Aamir Khan, Amole Gupte","[Darsheel Safary, Aamir Khan, Tisca Chopra, Vi..."
3,Dangal,2016,"Action, Biography, Drama",8.3,"183,495",$12.39M,Former wrestler Mahavir Singh Phogat and his t...,Nitesh Tiwari,"[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh..."
4,PK,2014,"Comedy, Drama, Sci-Fi",8.1,"180,136",$10.62M,An alien on Earth loses the only device he can...,Rajkumar Hirani,"[Aamir Khan, Anushka Sharma, Sanjay Dutt, Boma..."


In [206]:
# saving data in csv file
movie_DF.to_csv("Bollywood_movies.csv")

In [207]:
movie_DF.head(10)

,Name of movie,Year of release,Genre,Movie Rating,Votes,Gross,Summary,Director,Cast
0,The Kashmir Files,2022,Drama,8.3,"548,107",****,"The Kashmir Files' is a story, based on video ...",Vivek Agnihotri,"[Mithun Chakraborty, Anupam Kher, Darshan Kuma..."
1,3 Idiots,2009,"Comedy, Drama",8.4,"387,084",$6.53M,Two friends are searching for their long lost ...,Rajkumar Hirani,"[Aamir Khan, Madhavan, Mona Singh, Sharman Joshi]"
2,Taare Zameen Par,2007,"Drama, Family",8.3,"188,961",$1.22M,An eight-year-old boy is thought to be a lazy ...,"Directors:Aamir Khan, Amole Gupte","[Darsheel Safary, Aamir Khan, Tisca Chopra, Vi..."
3,Dangal,2016,"Action, Biography, Drama",8.3,"183,495",$12.39M,Former wrestler Mahavir Singh Phogat and his t...,Nitesh Tiwari,"[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh..."
4,PK,2014,"Comedy, Drama, Sci-Fi",8.1,"180,136",$10.62M,An alien on Earth loses the only device he can...,Rajkumar Hirani,"[Aamir Khan, Anushka Sharma, Sanjay Dutt, Boma..."
5,Valerian and the City of a Thousand Planets,2017,"Action, Adventure, Fantasy",6.4,"178,473",$41.19M,"A dark force threatens Alpha, a vast metropoli...",Luc Besson,"[Dane DeHaan, Cara Delevingne, Clive Owen, Rih..."
6,Radhe,2021,"Action, Crime, Thriller",1.9,"176,733",****,"After taking the dreaded gangster Gani Bhai, A...",Prabhu Deva,"[Salman Khan, Disha Patani, Randeep Hooda, Jac..."
7,Dil Bechara,2020,"Comedy, Drama, Romance",8.1,"128,821",****,The emotional journey of two hopelessly in lov...,Mukesh Chhabra,"[Sushant Singh Rajput, Sanjana Sanghi, Sahil V..."
8,Rang De Basanti,2006,"Comedy, Crime, Drama",8.1,"118,227",$2.20M,The story of six young Indians who assist an E...,Rakeysh Omprakash Mehra,"[Aamir Khan, Soha Ali Khan, Siddharth, Sharman..."
9,Shershaah,2021,"Action, Biography, Drama",8.4,"117,204",****,The story of PVC awardee Indian soldier Capt. ...,Vishnuvardhan,"[Sidharth Malhotra, Kiara Advani, Shiv Panditt..."


In [208]:
import jovian

In [209]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ········
[jovian] Committed successfully! https://jovian.ai/mdtausifc/bollywood-movie-scrapping


'https://jovian.ai/mdtausifc/bollywood-movie-scrapping'